In [6]:
import sys
!{sys.executable} -m pip install langchain==0.0.320 langchain_experimental==0.0.32 openai==0.28.1 rdflib neo4j==5.0.1 psycopg2-binary

In [7]:
from os import getcwd
from os.path import join, dirname
folder = dirname(getcwd())

In [8]:
import os
import json
from langchain.chat_models import ChatOpenAI

credentials = join(folder, "credentials.json")
with open(credentials) as f:
    data = json.load(f)
    key = data['OPENAI_API_KEY']
    
os.environ['OPENAI_API_KEY'] = key

In [9]:
# example procedure
keyboard_example = """
###
Clean a Computer Keyboard Do you notice that your computer keyboard looks 
dirty and needs a good cleaning? Or, you spilled something on it and need 
help to clean it properly without causing damage.nKeeping your keyboard 
clean not only improves its appearance but also helps to prolong its lifespan. 
In this article, we'll walk you through how to clean a computer keyboard effectively 
without causing any harm to the keys. A dirty keyboard can be a breeding ground for 
germs and bacteria and make typing a less pleasant experience. But before you grab 
the cleaning solution and start scrubbing away with the best keyboard cleaners, 
it's essential to know how to clean your keyboard correctly to avoid damaging it. 
What You'll Need Cleaning slime Keyboard brush Old toothbrush or baby bottle brush 
(optional) Compressed air Vacuum cleaner Cotton swabs Rubbing alcohol Toothpick (optional) 
Lint-free cloth or microfiber cloth Instructions Unplug and turn off your computer 
In case of an accident, you want to ensure nothing is harmed. Remove the batteries, 
if they are removable, from a wireless keyboard, and you should be set to go. 
Remove filth with a brush The abundance of dirty keyboards has led to the development of 
specialized computer keyboard brushes in various sizes and shapes that can remove the crumbs, 
hair, and other debris. However, you may clean your keyboard with everyday objects like a 
regular toothbrush or a used baby bottle brush. If you run the brush between the keys, your 
keyboard will be as spotless as the day you purchased it. That assumes the dirt isn't clung 
to the sides of the keys. Vacuum the keyboard using a small nozzle dust vacuum or a vacuum 
with a brush extension if you have one. It's essential to do this thoroughly because the 
vacuum's strong suction power can pull even the most stubborn debris. After vacuuming, use a 
cotton swab dipped in a rubbing alcohol to remove any remaining dust or grime. 
Pay attention not to accidentally wipe the paint off the keys
Polish with a lint-free cloth A lint-free cloth is strongly advised because you don't want to risk clogging 
the mechanical keyboard with additional dirt as you attempt to disinfect it. Don't use eccessive pressure 
Type Away to a Spotless 
Keyboard With These Easy Tips! Keeping your computer keyboard clean is essential to maintaining 
its functionality and longevity. Following the steps outlined in this article, you can effectively 
remove dirt and debris without causing damage to your keyboard. Regular cleaning ensures your keyboard 
will remain in top condition and provide a better typing experience. Make your bedroom a sanctuary again 
with the help of a trusted professional house cleaning service. Also, remember to check out our ultimate
guide for keeping your bedroom sparkling clean. 
###
"""

In [10]:
# example output for prompt json version
keyboard_json_output = """
{ 

"title": "Clean a Computer Keyboard",  

"steps": [  

{ "step_title": "Unplug and turn off your computer." , 

"order": "1" , 

"actions": "unplug;turn off" } ,  

{ "step_title": "Remove the batteries, if they are removable, from a wireless keyboard." , 

"order": "2" , 

"actions": "remove" } , 

{ "step_title": "Remove filth with a brush." , 

"order": "3" , 

"tools": "brush", 

"actions": "remove" } , 

{ "step_title": "Vacuum the keyboard using a small nozzle dust vacuum or a vacuum with a brush extension if you have one." , 

"order": "4", 

"tools": "nozzle dust vacuum;vacuum with a brush extension", 

"actions": "vacuum" } , 

{ "step_title": "Use a cotton swab dipped in a rubbing alcohol to remove any remaining dust or grime." , 

"order": "5" , 

"tools": "swab" , 

"actions": "remove" } , 

{ "step_title": "Polish with a lint-free cloth.", 

"order": "6" , 

"tools": "lint-free cloth" , 

"actions": "polish" }  

] 

}      
"""

In [11]:
procedure_for_llm_flat_monitor = """
###  
How to clean a flat panel monitor A flat panel monitor requires some extra care when cleaning. Since the surface is antistatic and softer than a glass panel CRT monitor, using improper cleaning methods can cause damage, such as scratches, etching or cloudiness The other parts of the monitor such as the frame and stand are also made of material that needs to be cleaned differently than other household items. Learn how to clean a flat panel monitor properly to protect your investment and maintain a crisp, clear view. Keep it clean to avoid dust and grime buildup. Purchase a soft cotton cleaning cloth for maintenance. Microfiber cloths designed for cleaning electronics are available at department stores, electronics departments and office supply stores. Other soft materials that can be used include soft, worn cotton or flannel. Paper towels or unwashed, new cloth is too abrasive for the surface. Keep the cleaning cloth in a dust free area. The cloth should be dedicated for use on the flat screen monitor only. Using it on other items can pick up dirt and debris that can scratch the surface or spread dirt. Remove any powder that appears on the monitor during shipping. This can easily be removed with the soft cloth. Use the cloth to wipe up smudges and fingerprints as needed. To keep away dust, establish a routine of wiping the monitor at least once a week. Wipe the cloth across the screen. Move it from one side to the other, in one direction, from the top of the screen to the bottom. Do not rub at an area of the screen or apply pressure. Dampen the cloth with water to clean more stubborn streaks or smudges. Wring it out thoroughly so that it is slightly damp, not wet Use this to clean the surface of the screen as well as the plastic casing of the monitor. Make a cleaning solution to clean the flat screen monitor.This will help to get rid of grime or marks that cannot be removed with dry wiping or a damp cloth. Mix 1 part diluted isopropyl alcohol to 1 part water Spray the solution on the cloth only, never directly on the monitor. Alternatively, you can purchase a commercial cleaner at an electronics or computer store. Use care around the monitor. Do not touch it with fingers, pens, pencils or other sharp objects. Be careful with food or liquid that could splash the monitor. Teach children not to touch the monitor. Tell others how to clean a flat panel monitor if you have other household members or outside cleaning help, make sure they know how to properly clean the monitor. Regular household cleaners should never be used on the monitor screen or frame most are too abrasive and can cause scratching or fading. Window cleaners like Windex contain ammonia which can cause etching and cloudiness. 
###  
"""

In [12]:
# prompt json version
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

template ="""
As an expert in information extraction, please, find all relevant steps of a procedure. 
A procedure consists of some steps that, if executed, contribute to achieve a goal.
A step of a procedure is an instruction that describes an action that someone needs to perform.
A procedure may contain, together with a list of steps, additional sentences that provide some tips, warnings, comments, tools to be used during the step, and that do not describe an action to be performed, thus they are not steps. 

A step should contain an action to perform, however, in some cases, even if including an action, some sentences are not steps, for example:

"do not do this" is not an actual step
"be careful not to do this" is not an actual step
"pay attention not to do this" is not an actual step
"you may want to do this" is not an actual step
"you may do this" is not an actual step

You have to number the steps, based on their order, and add the order number of each step in the output.
We also want to find the tools explicitly mentioned in all the steps we identified.
By tool I mean any kind of object needed to perform the action described in the sentence of the step.
Include the name of the tool or tools. Do not include any entry if no tool is explicitly mentioned. 
Finally, we want to find the actions explicitly mentioned in all the steps we identified.
The action should be expressed as a verb, and it is the action that the sentence of the step says to perform. A step should have at least one action.

Let's use an example: 

{keyboard_example}

Let's extract the steps, the actions and the tools, in json format:

{keyboard_json_output}

The example ends here. 

Extract steps, the actions and the tools from the following text, using the same json structure as in the example. 
{procedure_for_llm}
"""

prompt = PromptTemplate(
    input_variables=["keyboard_example","keyboard_json_output", "procedure_for_llm"],
    template = template                      
)

chain1 = LLMChain(
    llm=ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0),
    prompt=prompt,
    output_key="procedure-json"
)

In [13]:
# set procedure to be processed by LLM
procedure_for_llm = procedure_for_llm_flat_monitor

In [14]:
# final result in json
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[chain1],
    input_variables=["keyboard_example", "procedure_for_llm", "keyboard_json_output"],
    output_variables=["procedure-json"],
    verbose=True
)

output = overall_chain({"keyboard_example": keyboard_example,
           "keyboard_json_output": keyboard_json_output,
            "procedure_for_llm": procedure_for_llm})

print(output["procedure-json"])



> Entering new SequentialChain chain...

> Finished chain.
{
  "title": "How to clean a flat panel monitor",
  "steps": [
    {
      "step_title": "Purchase a soft cotton cleaning cloth for maintenance.",
      "order": "1",
      "tools": "soft cotton cleaning cloth",
      "actions": "purchase"
    },
    {
      "step_title": "Remove any powder that appears on the monitor during shipping.",
      "order": "2",
      "tools": "soft cloth",
      "actions": "remove"
    },
    {
      "step_title": "Use the cloth to wipe up smudges and fingerprints as needed.",
      "order": "3",
      "tools": "cloth",
      "actions": "wipe"
    },
    {
      "step_title": "Wipe the cloth across the screen. Move it from one side to the other, in one direction, from the top of the screen to the bottom.",
      "order": "4",
      "tools": "cloth",
      "actions": "wipe"
    },
    {
      "step_title": "Dampen the cloth with water to clean more stubborn streaks or smudges. Wring it out thorough